In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#--------models
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor

from catboost import CatBoostRegressor

from xgboost import XGBRegressor

#------Random Search cv
from sklearn.model_selection import RandomizedSearchCV

#-------Metrics for Evaluation
from sklearn.metrics import ( mean_absolute_error,
                             mean_squared_error,
                             r2_score,
                             )

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [3]:
df = pd.read_csv('./data.csv')

In [4]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   gender                       1000 non-null   object
 1   race_ethnicity               1000 non-null   object
 2   parental_level_of_education  1000 non-null   object
 3   lunch                        1000 non-null   object
 4   test_preparation_course      1000 non-null   object
 5   math_score                   1000 non-null   int64 
 6   reading_score                1000 non-null   int64 
 7   writing_score                1000 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 62.6+ KB


In [9]:
X = df.drop(columns=['math_score'], axis=1)
y = df['math_score']

In [10]:
X

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75
...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,99,95
996,male,group C,high school,free/reduced,none,55,55
997,female,group C,high school,free/reduced,completed,71,65
998,female,group D,some college,standard,completed,78,77


In [11]:
y

0      72
1      69
2      90
3      47
4      76
       ..
995    88
996    62
997    59
998    68
999    77
Name: math_score, Length: 1000, dtype: int64

In [15]:
num_feature = X.select_dtypes(exclude='object').columns
cat_features = X.select_dtypes(include='object').columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ('OneHotEncoder', oh_transformer, cat_features),
        ('StandardScaler', numeric_transformer, num_feature)
    ]
)

In [16]:
X = preprocessor.fit_transform(X)

In [17]:
X.shape

(1000, 19)

In [18]:
X

array([[ 1.        ,  0.        ,  0.        , ...,  1.        ,
         0.19399858,  0.39149181],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         1.42747598,  1.31326868],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.77010859,  1.64247471],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.12547206, -0.20107904],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.60515772,  0.58901542],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.15336989,  1.18158627]], shape=(1000, 19))

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( X,y, 
                                                    test_size=0.2, 
                                                    random_state=42)


In [20]:
def model_evaluator(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    rmse = np.sqrt(mse)
    
    return mae, mse, rmse, r2

In [26]:

models = {
    
    "Linear Regression": LinearRegression(),
    'Lasso': Lasso(),
    "Ridge": Ridge(),
    'knn': KNeighborsRegressor(),
    'Desion Tree': DecisionTreeRegressor(),
    'Random Forest': RandomForestRegressor(),
    "XGB": XGBRegressor(),
    "CatBoost": CatBoostRegressor(verbose=False),
    'AdamBoost': AdaBoostRegressor() 
}

model_list = []
r2_list = []

for model_name, model in models.items():
    
    model.fit(X_train, y_train) # Model Training
    
    y_train_pred = model.predict(X_train) # Model Prediction on Train Data
    y_test_pred = model.predict(X_test)   # Model prediction on test data
    
    # Model Evaluation
    
    train_mae, train_mse, train_rmse, train_r2 = model_evaluator(y_train, y_train_pred)
    test_mae, test_mse, test_rmse, test_r2     = model_evaluator(y_test, y_test_pred)
    
    model_list.append(model_name)
    r2_list.append(round(test_r2, 2))
    
    print(f"{model_name}: R2 Score = {test_r2} ")
    print("="*30)
    
    
    

Linear Regression: R2 Score = 0.8804332983749565 
Lasso: R2 Score = 0.8253197323627853 
Ridge: R2 Score = 0.8805931485028737 
knn: R2 Score = 0.7838129945787431 
Desion Tree: R2 Score = 0.7362931621149806 
Random Forest: R2 Score = 0.8517750067053398 
XGB: R2 Score = 0.8277965784072876 
CatBoost: R2 Score = 0.8516318920747058 
AdamBoost: R2 Score = 0.8502376847348861 


In [27]:
pd.DataFrame( 
             list(zip(model_list, r2_list)),
             columns=['Model_Name', "R2_Score"],
             ).sort_values(by=['R2_Score'], ascending=False)

,Model_Name,R2_Score
0,Linear Regression,0.88
2,Ridge,0.88
8,AdamBoost,0.85
7,CatBoost,0.85
5,Random Forest,0.85
1,Lasso,0.83
6,XGB,0.83
3,knn,0.78
4,Desion Tree,0.74


In [29]:
pd.DataFrame(
    {
        'Model_Name': model_list,
        'R2_Score': r2_list
    }
    
).sort_values(by=['R2_Score'], ascending=False)

,Model_Name,R2_Score
0,Linear Regression,0.88
2,Ridge,0.88
8,AdamBoost,0.85
7,CatBoost,0.85
5,Random Forest,0.85
1,Lasso,0.83
6,XGB,0.83
3,knn,0.78
4,Desion Tree,0.74
